# Introducción Apache Spark utilizando Pyspark 


## Importamos SparkSession

Las aplicaciones de PySpark comienzan con la inicialización, SparkSession que es el punto de entrada de PySpark como se muestra a continuación. En caso de ejecutarlo en el shell PySpark a través del ejecutable pyspark, el shell crea automáticamente la sesión en la variable spark para los usuarios.

*Nota:* Cargar al inicio todos los métodos/modulos que se usarán a lo largo del notebook.


In [52]:
import os
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

In [53]:
spark = (
    SparkSession.builder
    .appName("TestDataPlatform")
    .master("local[*]")
    .config('spark.jars', ','.join(os.path.join(dp, f) for dp, _, fs in os.walk('/home/jovyan/jars') for f in fs))
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.path.style.access", os.getenv("SPARK_S3_PATH_STYLE_ACCESS"))
    .config("spark.hadoop.fs.s3a.endpoint", os.getenv("SPARK_S3_ENDPOINT"))
    .config("spark.hadoop.fs.s3a.access.key", os.getenv("SPARK_S3_ACCESS_KEY"))
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv("SPARK_S3_SECRET_KEY"))
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.warehouse.dir", os.getenv("SPARK_HIVE_WAREHOUSE_DIR"))
    .config("hive.metastore.uris", os.getenv("SPARK_HIVE_METASTORE_URIS"))
    .enableHiveSupport()
)
spark = configure_spark_with_delta_pip(spark).getOrCreate()

In [54]:
spark

#### Terminamos la sesión actual
No podemos tener mas de una sesión a la vez en nuestro notebook, por lo cual con el método 'stop' terminaremos la applicación.

De la misma forma, al terminar una applicación, debemos de indicar explicitamente que termine. De otra forma no liberará los recursos asignados.

In [ ]:
spark.stop()

# RDD (Resilient Distributed Dataset)
Un RDD, según Spark, se define como una colección de elementos que es tolerante a fallos y que es capaz de operar en paralelo.
Es importante recalcar el tema de que sea capaz de operar en paralelo, porque es la clave o la filosofía básica de Apache Spark.

Los RDDs tienen como características principales las siguientes:

* Es la principal abstracción de datos, el tipo de dato básico que tiene Apache Spark.
* Los RDD están particionados en los distintos nodos del clúster, ya que Apache Spark se suele instalar en un clúster o conjunto de máquinas, por lo que esos RRDs se encuentran distribuidos sobre esas máquinas. Con ello se consigue la tolerancia a fallos, porque si falla una máquina tenemos el fichero en otras máquinas.
* Se suelen crear a partir de un fichero del HDFS, el sistema de ficheros distribuidos de Hadoop.
* Usan la evaluación perezosa, que consiste en que todas las transformaciones que vayamos haciendo a los RDDs se van a ir almacenando en un DAG y no se van a resolver hasta que no quede más remedio, hasta que la herramienta esté obligada a realizarlas. Esta evaluación perezosa tiene una ventaja y un inconveniente, la primera es que ganamos tiempo, y el inconveniente es que si falla, no lo vamos a ver hasta que se resuelva el grafo completo.

In [6]:
#creamos una lista
lista = ['Apache Spark', 'Curso de Introducción', 'Big data', 'Cloud']

In [7]:
#Crear RDD utilizando parallelize   

rdd=spark.sparkContext.parallelize(lista)

In [8]:
rdd.collect()

['Apache Spark', 'Curso de Introducción', 'Big data', 'Cloud']

##### flatMap() 
transformación aplana el RDD después de aplicar la función y devuelve un nuevo RDD. En el siguiente ejemplo, primero, divide cada registro por espacio en un RDD y finalmente lo aplana. El RDD resultante consta de una sola palabra en cada registro.

##### map()
transformación se utiliza para aplicar operaciones complejas como agregar una columna, actualizar una columna, etc., la salida de las transformaciones del mapa siempre tendrá el mismo número de registros que la entrada.

En nuestro ejemplo de recuento de palabras, estamos agregando una nueva columna con valor 1 para cada palabra, el resultado del RDD es PairRDDFunctionsque contiene pares clave-valor, palabra de tipo Cadena como clave y 1 de tipo Int como valor.

##### reduceByKey()
fusiona los valores de cada clave con la función especificada. En nuestro ejemplo, reduce la cadena de palabras aplicando la función de suma en valor. El resultado de nuestro RDD contiene palabras únicas y su recuento. 

##### sortByKey()
transformación se utiliza para ordenar elementos RDD en clave. En nuestro ejemplo, primero, convertimos RDD [(String, Int]) a RDD [(Int, String]) usando la transformación de mapa y aplicamos sortByKey que idealmente ordena en un valor entero. Y finalmente, foreach con declaraciones println devuelve todas las palabras en RDD y su recuento como par clave-valor

##### filter() 
se utiliza para filtrar los registros en un RDD. En nuestro ejemplo, estamos filtrando todas las palabras que comienzan con "a".
##### first()
devuelve el primer registro.
##### max()
devuelve el registro máximo.

##### reduce() 
reduce los registros a uno solo, podemos usar esto para contar o sumar.

## DataFrame

Un DataFrame es un DataSet que a la vez está organizado en columnas.
Un DataSet es una colección de datos distribuidos que tienen ya una estructura, a diferencia de los RDD, que son conjuntos de datos desestructurados.
Vamos a tener los datos estructurados y cada columna con su nombre correspondiente, con lo que nos va a resultar mucho más sencillo consultar, modificar o transformar ese conjunto de datos.

Se pueden crear PySpark DataFrame a partir de fuentes de datos como TXT, CSV, JSON, ORV, Avro, Parquet, formatos XML leyendo desde HDFS, S3, DBFS, sistemas de archivos Azure Blob, etc.

In [9]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

dataset = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=dataset,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



#### Cree DataFrame a partir de fuentes de datos

En tiempo real, la mayoría de las veces crea DataFrame a partir de archivos de origen de datos como CSV, Text, JSON, XML, etc.

PySpark de forma predeterminada admite muchos formatos de datos listos para usar sin importar ninguna biblioteca y para crear DataFrame debe usar el método apropiado disponible en la DataFrameReaderclase.

##### csv
tilice el csv()método del DataFrameReaderobjeto para crear un DataFrame a partir de un archivo CSV. también puede proporcionar opciones como qué delimitador usar, si ha citado datos, formatos de fecha, esquema de inferir

<code> df2 = spark.read.csv("/src/resources/file.csv") </code>

##### txt
De manera similar, también puede crear un DataFrame leyendo un archivo de texto, use el text()método del DataFrameReader para hacerlo.

<code> df2 = spark.read.text("/src/resources/file.txt") </code>
#####  JSON

PySpark también se utiliza para procesar archivos de datos semiestructurados como el formato JSON. puede usar el json()método del DataFrameReader para leer el archivo JSON en DataFrame. A continuación se muestra un ejemplo sencillo.

<code> df2 = spark.read.json("/src/resources/file.json") </code>

##### Otras fuentes (Avro, Parquet, ORC, Kafka)
También podemos crear DataFrame leyendo Avro, Parquet, ORC, archivos binarios y accediendo a la tabla Hive y HBase, y también leyendo datos de Kafka que he explicado en los artículos a continuación, recomendaría leerlos cuando tenga tiempo.

<code> parDF1=spark.read.parquet("/temp/out/people.parquet")</code>


#### Ejemplo
Creando un dataframe desde un csv almacenado en cloud storage 

In [57]:
from pyspark.sql.types import *
ruta = 's3a://lhchdev/datalake/empleado.csv'

df_schema = StructType([
StructField("ID", StringType(),True),
StructField("NOMBRE", StringType(),True),
StructField("TELEFONO", StringType(),True),
StructField("CORREO", StringType(),True),
StructField("FECHA_INGRESO", StringType(),True),
StructField("EDAD", IntegerType(),True),
StructField("SALARIO", DoubleType(),True),
StructField("ID_EMPRESA", StringType(),True),
])


df = spark.read.format("CSV").option("header","true").option("delimiter",",").schema(df_schema).load(ruta)

## FUNCIONES
#### show()
muestra el contenido de DataFrame en la tabla.

se utiliza para mostrar el contenido del DataFrame en un formato de fila y columna de tabla. De forma predeterminada, muestra solo 20 filas y los valores de las columnas se truncan a los 20 caracteres

* Sintaxis
 <code> def show(self, n=20, truncate=True, vertical=False) </code>

In [58]:
df.show(20)

+---+-------------------+-----------------+--------------------+-------------+----+-------+----------+
| ID|             NOMBRE|         TELEFONO|              CORREO|FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+-------------------+-----------------+--------------------+-------------+----+-------+----------+
|353|      Mateo Mendoza| +56 89 7366-3177|mateo.mendozalive...|   2014-11-23|  29|55560.0|         6|
|690|        Bert Torres|+507 73 8501-1586|bert.torres@terra.cl|   2019-10-19|  23|13196.0|         3|
|486|         Jonah Vera| +593 903-986-474|jonah.vera@uol.co...|   2020-11-27|   0|23455.0|         6|
|389|        Jonah Rojas|+593 25 6426-6907|jonah.rojas@gmail...|   2016-03-17|  65|20797.0|         9|
| 32|      Carl Martinez|+502 46 1534-4792|carl.martinez@out...|   2012-12-03|  63|10764.0|         8|
|443|      Cadman Torres| +502 9 8945-4288|cadman.torres@liv...|   2005-02-22|  22| 1501.0|         3|
|199|      Cadman Huanca|  +52 939-904-918|cadman.huanca@out...|   2022-0

### select()
función se usa para seleccionar una, varias, columna por índice, todas las columnas de la lista y las columnas anidadas de un DataFrame, PySpark select () 
Es una función de transformación, por lo que devuelve un nuevo DataFrame con las columnas seleccionadas.

In [27]:
# seleccionar NOMBRE y CORREO

df.select("NOMBRE", "CORREO").show(1)

df.select(df.NOMBRE, df.CORREO).show(1)

df.select(df["NOMBRE"], df["CORREO"]).show(1)

from pyspark.sql.functions import col

df.select(col("NOMBRE"), col("CORREO")).show(1)


+------+--------------------+
|NOMBRE|              CORREO|
+------+--------------------+
|  Carl|arcu.Sed.et@ante....|
+------+--------------------+
only showing top 1 row

+------+--------------------+
|NOMBRE|              CORREO|
+------+--------------------+
|  Carl|arcu.Sed.et@ante....|
+------+--------------------+
only showing top 1 row

+------+--------------------+
|NOMBRE|              CORREO|
+------+--------------------+
|  Carl|arcu.Sed.et@ante....|
+------+--------------------+
only showing top 1 row

+------+--------------------+
|NOMBRE|              CORREO|
+------+--------------------+
|  Carl|arcu.Sed.et@ante....|
+------+--------------------+
only showing top 1 row



#### Con expresiones regulares (colRegex)

Recuerda: usar comillas invertidas alrededor del patrón.

In [30]:
# 1) Columnas que empiezan con "ID" (capta ID e ID_EMPRESA)
df.select(df.colRegex("`^ID(_.*)?$`")).show(2)

# 2) Columnas que contienen "FECHA" (FECHA_INGRESO)
df.select(df.colRegex("`^FECHA_.*`")).show(2)

# 3) Columnas que terminan en "O" (TELEFONO, CORREO, SALARIO)
df.select(df.colRegex("`^.*O$`")).show(2)

# 4) Columnas relacionadas a "contacto": NOMBRE | TELEFONO | CORREO
df.select(df.colRegex("`^(NOMBRE|TELEFONO|CORREO)$`")).show(2)


+---+----------+
| ID|ID_EMPRESA|
+---+----------+
|  1|         5|
|  2|         2|
+---+----------+
only showing top 2 rows

+-------------+
|FECHA_INGRESO|
+-------------+
|   2004-04-23|
|   2019-02-17|
+-------------+
only showing top 2 rows

+--------------+--------------------+-------------+-------+
|      TELEFONO|              CORREO|FECHA_INGRESO|SALARIO|
+--------------+--------------------+-------------+-------+
|1-745-633-9145|arcu.Sed.et@ante....|   2004-04-23|20095.0|
|      155-2498|Donec.egestas.Ali...|   2019-02-17| 9298.0|
+--------------+--------------------+-------------+-------+
only showing top 2 rows

+---------+--------------+--------------------+
|   NOMBRE|      TELEFONO|              CORREO|
+---------+--------------+--------------------+
|     Carl|1-745-633-9145|arcu.Sed.et@ante....|
|Priscilla|      155-2498|Donec.egestas.Ali...|
+---------+--------------+--------------------+
only showing top 2 rows



In [31]:
from pyspark.sql.functions import year, month

# Renombrar y derivar campos
df.select(
    col("ID"),
    col("NOMBRE").alias("nombre"),
    col("TELEFONO").alias("telefono"),
    col("CORREO").alias("correo"),
    year(col("FECHA_INGRESO")).alias("anio_ingreso"),
    month(col("FECHA_INGRESO")).alias("mes_ingreso"),
    col("EDAD"),
    col("SALARIO"),
    col("ID_EMPRESA")
).show()


+---+---------+--------------+--------------------+------------+-----------+----+-------+----------+
| ID|   nombre|      telefono|              correo|anio_ingreso|mes_ingreso|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+--------------------+------------+-----------+----+-------+----------+
|  1|     Carl|1-745-633-9145|arcu.Sed.et@ante....|        2004|          4|  32|20095.0|         5|
|  2|Priscilla|      155-2498|Donec.egestas.Ali...|        2019|          2|  34| 9298.0|         2|
|  3|  Jocelyn|1-204-956-8594|amet.diam@loborti...|        2002|          8|  27|10853.0|         3|
|  4|    Aidan|1-719-862-9385|euismod.et.commod...|        2018|         11|  29| 3387.0|        10|
|  5|  Leandra|      839-8044|at@pretiumetrutru...|        2002|         10|  41|22102.0|         1|
|  6|     Bert|      797-4453|a.felis.ullamcorp...|        2017|          4|  70| 7800.0|         7|
|  7|     Mark|1-680-102-6792|Quisque.ac@placer...|        2006|          4|  52| 8112.0|  

In [32]:
# Registrar la tabla como vista temporal
df.createOrReplaceTempView("empleados")

# Ahora puedes ejecutar consultas SQL sobre ella
result = spark.sql("""
    SELECT 
        ID,
        NOMBRE,
        CORREO,
        TELEFONO,
        SALARIO,
        ID_EMPRESA
    FROM empleados
    WHERE EDAD > 30
    ORDER BY SALARIO DESC
""")

result.show()


+---+--------+--------------------+--------------+-------+----------+
| ID|  NOMBRE|              CORREO|      TELEFONO|SALARIO|ID_EMPRESA|
+---+--------+--------------------+--------------+-------+----------+
| 82|   Cally|        id@dolor.edu|1-658-890-5167|24575.0|        10|
| 34|    Lila|     viverra@sit.edu|1-703-777-4150|24305.0|         4|
| 95|   Jayme|ornare.Fusce@tinc...|1-641-113-8418|23975.0|         4|
| 66|  Adrian|enim@mollisPhasel...|      220-8905|22953.0|         2|
| 39| Carolyn|metus.Aenean.sed@...|      846-7060|22838.0|         6|
|  5| Leandra|at@pretiumetrutru...|      839-8044|22102.0|         1|
| 33|     Jin|est.Nunc.ullamcor...|1-620-779-3366|22038.0|         2|
| 31|   Rylee| Sed.nunc@turpis.edu|      306-5447|21591.0|         3|
| 43|   Yetta|vitae@dapibusrutr...|      986-0220|21452.0|         2|
| 26| Brenden|elit.pede.malesua...|1-455-726-9413|20549.0|         7|
|  1|    Carl|arcu.Sed.et@ante....|1-745-633-9145|20095.0|         5|
| 41|   Wynne|eget@d

#### withColumn()
Es una función de transformación de DataFrame que se utiliza para cambiar el valor, convertir el tipo de datos de una columna existente, crear una nueva columna y muchos más. En esta publicación, lo guiaré a través de las operaciones de columna de PySpark DataFrame de uso común usando ejemplos withColumn ().

##### 1. Convertir el tipo de dato de una columna (cast)

Cambia el tipo de SALARIO a Integer:

In [33]:
from pyspark.sql.functions import col, lit

df2 = df.withColumn("SALARIO", col("SALARIO").cast("Integer"))
df2.printSchema()
df2.show(truncate=False)


root
 |-- ID: string (nullable = true)
 |-- NOMBRE: string (nullable = true)
 |-- TELEFONO: string (nullable = true)
 |-- CORREO: string (nullable = true)
 |-- FECHA_INGRESO: string (nullable = true)
 |-- EDAD: integer (nullable = true)
 |-- SALARIO: integer (nullable = true)
 |-- ID_EMPRESA: string (nullable = true)

+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+
|ID |NOMBRE   |TELEFONO      |CORREO                                 |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+
|1  |Carl     |1-745-633-9145|arcu.Sed.et@ante.co.uk                 |2004-04-23   |32  |20095  |5         |
|2  |Priscilla|155-2498      |Donec.egestas.Aliquam@volutpatnunc.edu |2019-02-17   |34  |9298   |2         |
|3  |Jocelyn  |1-204-956-8594|amet.diam@lobortis.co.uk               |2002-08-01   |27  |10853  |3         |
|4  |Aidan    |1-719-862-9

#### 💰 2. Operar sobre una columna numérica

Multiplicar el salario por 1.1 (aumentar 10%):

In [34]:
df3 = df.withColumn("SALARIO", col("SALARIO") * 1.1)
df3.printSchema()
df3.show(truncate=False)


root
 |-- ID: string (nullable = true)
 |-- NOMBRE: string (nullable = true)
 |-- TELEFONO: string (nullable = true)
 |-- CORREO: string (nullable = true)
 |-- FECHA_INGRESO: string (nullable = true)
 |-- EDAD: integer (nullable = true)
 |-- SALARIO: double (nullable = true)
 |-- ID_EMPRESA: string (nullable = true)

+---+---------+--------------+---------------------------------------+-------------+----+------------------+----------+
|ID |NOMBRE   |TELEFONO      |CORREO                                 |FECHA_INGRESO|EDAD|SALARIO           |ID_EMPRESA|
+---+---------+--------------+---------------------------------------+-------------+----+------------------+----------+
|1  |Carl     |1-745-633-9145|arcu.Sed.et@ante.co.uk                 |2004-04-23   |32  |22104.5           |5         |
|2  |Priscilla|155-2498      |Donec.egestas.Aliquam@volutpatnunc.edu |2019-02-17   |34  |10227.800000000001|2         |
|3  |Jocelyn  |1-204-956-8594|amet.diam@lobortis.co.uk               |2002-08-01 

In [35]:
df3 = df.withColumn("SALARIO", col("SALARIO") * 1.1)
df3.printSchema()
df3.show(truncate=False)


root
 |-- ID: string (nullable = true)
 |-- NOMBRE: string (nullable = true)
 |-- TELEFONO: string (nullable = true)
 |-- CORREO: string (nullable = true)
 |-- FECHA_INGRESO: string (nullable = true)
 |-- EDAD: integer (nullable = true)
 |-- SALARIO: double (nullable = true)
 |-- ID_EMPRESA: string (nullable = true)

+---+---------+--------------+---------------------------------------+-------------+----+------------------+----------+
|ID |NOMBRE   |TELEFONO      |CORREO                                 |FECHA_INGRESO|EDAD|SALARIO           |ID_EMPRESA|
+---+---------+--------------+---------------------------------------+-------------+----+------------------+----------+
|1  |Carl     |1-745-633-9145|arcu.Sed.et@ante.co.uk                 |2004-04-23   |32  |22104.5           |5         |
|2  |Priscilla|155-2498      |Donec.egestas.Aliquam@volutpatnunc.edu |2019-02-17   |34  |10227.800000000001|2         |
|3  |Jocelyn  |1-204-956-8594|amet.diam@lobortis.co.uk               |2002-08-01 

#### 📋 3. Crear una nueva columna a partir de otra

Crear columna SALARIO_NEGATIVO (versión negativa del salario):

In [36]:
df4 = df.withColumn("SALARIO_NEGATIVO", col("SALARIO") * -1)
df4.printSchema()
df4.show(truncate=False)

root
 |-- ID: string (nullable = true)
 |-- NOMBRE: string (nullable = true)
 |-- TELEFONO: string (nullable = true)
 |-- CORREO: string (nullable = true)
 |-- FECHA_INGRESO: string (nullable = true)
 |-- EDAD: integer (nullable = true)
 |-- SALARIO: double (nullable = true)
 |-- ID_EMPRESA: string (nullable = true)
 |-- SALARIO_NEGATIVO: double (nullable = true)

+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+----------------+
|ID |NOMBRE   |TELEFONO      |CORREO                                 |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|SALARIO_NEGATIVO|
+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+----------------+
|1  |Carl     |1-745-633-9145|arcu.Sed.et@ante.co.uk                 |2004-04-23   |32  |20095.0|5         |-20095.0        |
|2  |Priscilla|155-2498      |Donec.egestas.Aliquam@volutpatnunc.edu |2019-02-17   |34  |9298.0 |2         |-9298.0         |
|3 

#### 🌎 4. Agregar una columna constante con lit()

Agregar la columna PAIS con valor fijo "PERU":

In [38]:
df5 = df.withColumn("PAIS", lit("PERU"))
df5.printSchema()
df5.show(truncate=False)


root
 |-- ID: string (nullable = true)
 |-- NOMBRE: string (nullable = true)
 |-- TELEFONO: string (nullable = true)
 |-- CORREO: string (nullable = true)
 |-- FECHA_INGRESO: string (nullable = true)
 |-- EDAD: integer (nullable = true)
 |-- SALARIO: double (nullable = true)
 |-- ID_EMPRESA: string (nullable = true)
 |-- PAIS: string (nullable = false)

+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+----+
|ID |NOMBRE   |TELEFONO      |CORREO                                 |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|PAIS|
+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+----+
|1  |Carl     |1-745-633-9145|arcu.Sed.et@ante.co.uk                 |2004-04-23   |32  |20095.0|5         |PERU|
|2  |Priscilla|155-2498      |Donec.egestas.Aliquam@volutpatnunc.edu |2019-02-17   |34  |9298.0 |2         |PERU|
|3  |Jocelyn  |1-204-956-8594|amet.diam@lobortis.co.uk               |2002

#### 🧱 5. Agregar múltiples columnas nuevas

Agregar PAIS y CATEGORIA_EMPLEADO:

In [39]:
df6 = df.withColumn("PAIS", lit("PERU")) \
        .withColumn("CATEGORIA_EMPLEADO", lit("NOMINA"))
df6.printSchema()
df6.show(truncate=False)


root
 |-- ID: string (nullable = true)
 |-- NOMBRE: string (nullable = true)
 |-- TELEFONO: string (nullable = true)
 |-- CORREO: string (nullable = true)
 |-- FECHA_INGRESO: string (nullable = true)
 |-- EDAD: integer (nullable = true)
 |-- SALARIO: double (nullable = true)
 |-- ID_EMPRESA: string (nullable = true)
 |-- PAIS: string (nullable = false)
 |-- CATEGORIA_EMPLEADO: string (nullable = false)

+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+----+------------------+
|ID |NOMBRE   |TELEFONO      |CORREO                                 |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|PAIS|CATEGORIA_EMPLEADO|
+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+----+------------------+
|1  |Carl     |1-745-633-9145|arcu.Sed.et@ante.co.uk                 |2004-04-23   |32  |20095.0|5         |PERU|NOMINA            |
|2  |Priscilla|155-2498      |Donec.egestas.Aliquam@volutpatn

#### ✏️ 6. Renombrar columnas

Cambiar el nombre de NOMBRE → NOMBRE_COMPLETO:

In [40]:
df_renamed = df.withColumnRenamed("NOMBRE", "NOMBRE_COMPLETO")
df_renamed.show(truncate=False)


+---+---------------+--------------+---------------------------------------+-------------+----+-------+----------+
|ID |NOMBRE_COMPLETO|TELEFONO      |CORREO                                 |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------------+--------------+---------------------------------------+-------------+----+-------+----------+
|1  |Carl           |1-745-633-9145|arcu.Sed.et@ante.co.uk                 |2004-04-23   |32  |20095.0|5         |
|2  |Priscilla      |155-2498      |Donec.egestas.Aliquam@volutpatnunc.edu |2019-02-17   |34  |9298.0 |2         |
|3  |Jocelyn        |1-204-956-8594|amet.diam@lobortis.co.uk               |2002-08-01   |27  |10853.0|3         |
|4  |Aidan          |1-719-862-9385|euismod.et.commodo@nibhlaciniaorci.edu |2018-11-06   |29  |3387.0 |10        |
|5  |Leandra        |839-8044      |at@pretiumetrutrum.com                 |2002-10-10   |41  |22102.0|1         |
|6  |Bert           |797-4453      |a.felis.ullamcorper@arcu.org           |2017

#### 🗑️ 7. Eliminar una columna

Eliminar SALARIO_NEGATIVO:

In [41]:
df4.drop("SALARIO_NEGATIVO").show(truncate=False)


+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+
|ID |NOMBRE   |TELEFONO      |CORREO                                 |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+
|1  |Carl     |1-745-633-9145|arcu.Sed.et@ante.co.uk                 |2004-04-23   |32  |20095.0|5         |
|2  |Priscilla|155-2498      |Donec.egestas.Aliquam@volutpatnunc.edu |2019-02-17   |34  |9298.0 |2         |
|3  |Jocelyn  |1-204-956-8594|amet.diam@lobortis.co.uk               |2002-08-01   |27  |10853.0|3         |
|4  |Aidan    |1-719-862-9385|euismod.et.commodo@nibhlaciniaorci.edu |2018-11-06   |29  |3387.0 |10        |
|5  |Leandra  |839-8044      |at@pretiumetrutrum.com                 |2002-10-10   |41  |22102.0|1         |
|6  |Bert     |797-4453      |a.felis.ullamcorper@arcu.org           |2017-04-25   |70  |7800.0 |7         |
|7  |Mark     |1-68

#### 🔍 Extra opcional: Crear columna derivada de fecha

Obtener el año de ingreso:

In [42]:
from pyspark.sql.functions import year

df7 = df.withColumn("ANIO_INGRESO", year(col("FECHA_INGRESO")))
df7.show(truncate=False)


+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+------------+
|ID |NOMBRE   |TELEFONO      |CORREO                                 |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|ANIO_INGRESO|
+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+------------+
|1  |Carl     |1-745-633-9145|arcu.Sed.et@ante.co.uk                 |2004-04-23   |32  |20095.0|5         |2004        |
|2  |Priscilla|155-2498      |Donec.egestas.Aliquam@volutpatnunc.edu |2019-02-17   |34  |9298.0 |2         |2019        |
|3  |Jocelyn  |1-204-956-8594|amet.diam@lobortis.co.uk               |2002-08-01   |27  |10853.0|3         |2002        |
|4  |Aidan    |1-719-862-9385|euismod.et.commodo@nibhlaciniaorci.edu |2018-11-06   |29  |3387.0 |10        |2018        |
|5  |Leandra  |839-8044      |at@pretiumetrutrum.com                 |2002-10-10   |41  |22102.0|1         |2002        |
|6  |Bert     |797-4453 

#### ⚙️ 1️⃣ Ejemplo simple: clasificación por salario

Queremos crear una nueva columna llamada **`NIVEL_SALARIAL`** con la siguiente lógica:

| Condición                  | Resultado |
|-----------------------------|------------|
| `SALARIO >= 15000`          | **"ALTO"** |
| `SALARIO entre 8000 y 14999`| **"MEDIO"** |
| `SALARIO < 8000`            | **"BAJO"** |


In [43]:
from pyspark.sql.functions import when, col

df_case = df.withColumn(
    "NIVEL_SALARIAL",
    when(col("SALARIO") >= 15000, "ALTO")
    .when((col("SALARIO") >= 8000) & (col("SALARIO") < 15000), "MEDIO")
    .otherwise("BAJO")
)

df_case.select("NOMBRE", "SALARIO", "NIVEL_SALARIAL").show(truncate=False)


+---------+-------+--------------+
|NOMBRE   |SALARIO|NIVEL_SALARIAL|
+---------+-------+--------------+
|Carl     |20095.0|ALTO          |
|Priscilla|9298.0 |MEDIO         |
|Jocelyn  |10853.0|MEDIO         |
|Aidan    |3387.0 |BAJO          |
|Leandra  |22102.0|ALTO          |
|Bert     |7800.0 |BAJO          |
|Mark     |8112.0 |MEDIO         |
|Jonah    |17040.0|ALTO          |
|Hanae    |6834.0 |BAJO          |
|Cadman   |7996.0 |BAJO          |
|Melyssa  |4913.0 |BAJO          |
|Tanner   |19943.0|ALTO          |
|Trevor   |9501.0 |MEDIO         |
|Allen    |16289.0|ALTO          |
|Wanda    |1539.0 |BAJO          |
|Alden    |3377.0 |BAJO          |
|Omar     |6851.0 |BAJO          |
|Owen     |4759.0 |BAJO          |
|Laura    |17403.0|ALTO          |
|Emery    |18752.0|ALTO          |
+---------+-------+--------------+
only showing top 20 rows



#### filter()
función PySpark se usa para filtrar las filas de RDD / DataFrame según la condición dada o la expresión SQL, también puede usar una where() cláusula en lugar del filter() si proviene de un entorno SQL, ambas funciones operan exactamente igual.

In [45]:
from pyspark.sql.functions import col, lower

# ============================================================
# 1. Condiciones simples
# ============================================================

# Igualdad
df.filter(col("ID_EMPRESA") == 5).show(truncate=False)

# Desigualdad
df.filter(col("EDAD") != 30).show(truncate=False)

# Mayor / menor que
df.filter(col("SALARIO") > 15000).show(truncate=False)
df.filter(col("EDAD") < 25).show(truncate=False)

# Entre valores
df.filter(col("SALARIO").between(9000, 18000)).show(truncate=False)

# LIKE
df.filter(col("NOMBRE").like("%an%")).show(truncate=False)

# ============================================================
# 2. Condiciones múltiples (AND / OR)
# ============================================================

# Dos condiciones AND
df.filter((col("ID_EMPRESA") == 5) & (col("EDAD") >= 30)).show(truncate=False)

# Dos condiciones OR
df.filter((col("SALARIO") > 20000) | (col("EDAD") < 25)).show(truncate=False)

# Tres condiciones combinadas
df.filter(
    (col("SALARIO") > 8000) &
    (col("SALARIO") < 20000) &
    (col("EDAD") > 25)
).show(truncate=False)

# Combinación de AND / OR
df.filter(
    ((col("ID_EMPRESA") == 5) & (col("EDAD") >= 30)) |
    (col("SALARIO") > 20000)
).show(truncate=False)

# ============================================================
# 3. Condiciones con IN / NOT IN
# ============================================================

empresas = [2, 5, 7]

# IN
df.filter(col("ID_EMPRESA").isin(empresas)).show(truncate=False)

# NOT IN
df.filter(~col("ID_EMPRESA").isin(empresas)).show(truncate=False)

# IN + condición adicional
df.filter(col("ID_EMPRESA").isin(empresas) & (col("EDAD") > 40)).show(truncate=False)

# ============================================================
# 4. Expresiones con patrones
# ============================================================

# LIKE
df.filter(col("CORREO").like("%@vel.org")).show(truncate=False)

# RLIKE (regex)
df.filter(col("TELEFONO").rlike("^1-.*")).show(truncate=False)

# ILIKE (case-insensitive)
df.filter(lower(col("NOMBRE")).like("%an%")).show(truncate=False)


+---+--------+--------------+------------------------------------+-------------+----+-------+----------+
|ID |NOMBRE  |TELEFONO      |CORREO                              |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+--------+--------------+------------------------------------+-------------+----+-------+----------+
|1  |Carl    |1-745-633-9145|arcu.Sed.et@ante.co.uk              |2004-04-23   |32  |20095.0|5         |
|7  |Mark    |1-680-102-6792|Quisque.ac@placerat.ca              |2006-04-21   |52  |8112.0 |5         |
|8  |Jonah   |214-2975      |eu.ultrices.sit@vitae.ca            |2017-10-07   |23  |17040.0|5         |
|13 |Trevor  |512-1955      |Nunc.quis.arcu@egestasa.org         |2010-08-06   |34  |9501.0 |5         |
|15 |Wanda   |359-6973      |Nam.nulla.magna@In.org              |2005-08-21   |27  |1539.0 |5         |
|35 |Aurora  |1-865-751-3479|magna@Cras.net                      |2017-10-21   |54  |4588.0 |5         |
|50 |Ross    |1-587-285-1837|at.risus@milaciniamattis.c

In [49]:
# Crear vista temporal
df.createOrReplaceTempView("empleados")

# ============================================================
# 1. Condiciones simples
# ============================================================

spark.sql("""
  SELECT * FROM empleados
  WHERE ID_EMPRESA = 5
""").show(truncate=False)

spark.sql("""
  SELECT * FROM empleados
  WHERE SALARIO > 15000
""").show(truncate=False)

spark.sql("""
  SELECT * FROM empleados
  WHERE EDAD < 25
""").show(truncate=False)

spark.sql("""
  SELECT * FROM empleados
  WHERE SALARIO BETWEEN 9000 AND 18000
""").show(truncate=False)

spark.sql("""
  SELECT * FROM empleados
  WHERE NOMBRE LIKE '%an%'
""").show(truncate=False)

# ============================================================
# 2. Condiciones múltiples (AND / OR)
# ============================================================

# Dos condiciones (AND)
spark.sql("""
  SELECT * FROM empleados
  WHERE ID_EMPRESA = 5 AND EDAD >= 30
""").show(truncate=False)

# Tres condiciones (AND)
spark.sql("""
  SELECT * FROM empleados
  WHERE SALARIO > 8000 AND SALARIO < 20000 AND EDAD > 25
""").show(truncate=False)

# Combinadas (AND / OR)
spark.sql("""
  SELECT * FROM empleados
  WHERE (ID_EMPRESA = 5 AND EDAD >= 30) OR SALARIO > 20000
""").show(truncate=False)

# ============================================================
# 3. IN / NOT IN
# ============================================================

spark.sql("""
  SELECT * FROM empleados
  WHERE ID_EMPRESA IN (2,5,7)
""").show(truncate=False)

spark.sql("""
  SELECT * FROM empleados
  WHERE ID_EMPRESA NOT IN (2,5,7)
""").show(truncate=False)

spark.sql("""
  SELECT * FROM empleados
  WHERE ID_EMPRESA IN (2,5,7) AND EDAD > 40
""").show(truncate=False)

# ============================================================
# 4. LIKE / RLIKE
# ============================================================

spark.sql("""
  SELECT * FROM empleados
  WHERE CORREO LIKE '%@vel.org'
""").show(truncate=False)

spark.sql("""
  SELECT * FROM empleados
  WHERE TELEFONO RLIKE '^1-.*'
""").show(truncate=False)

spark.sql("""
  SELECT * FROM empleados
  WHERE lower(NOMBRE) LIKE '%an%'
""").show(truncate=False)

# ============================================================
# 5. Ejemplo avanzado con 4 condiciones
# ============================================================

spark.sql("""
  SELECT * FROM empleados
  WHERE (SALARIO > 10000 AND SALARIO < 20000)
    AND (EDAD BETWEEN 25 AND 50)
    AND (ID_EMPRESA IN (5,7))
    AND (CORREO LIKE '%@ante.co.uk' OR CORREO LIKE '%@dmagnaet.ca')
""").show(truncate=False)


+---+--------+--------------+------------------------------------+-------------+----+-------+----------+
|ID |NOMBRE  |TELEFONO      |CORREO                              |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+--------+--------------+------------------------------------+-------------+----+-------+----------+
|1  |Carl    |1-745-633-9145|arcu.Sed.et@ante.co.uk              |2004-04-23   |32  |20095.0|5         |
|7  |Mark    |1-680-102-6792|Quisque.ac@placerat.ca              |2006-04-21   |52  |8112.0 |5         |
|8  |Jonah   |214-2975      |eu.ultrices.sit@vitae.ca            |2017-10-07   |23  |17040.0|5         |
|13 |Trevor  |512-1955      |Nunc.quis.arcu@egestasa.org         |2010-08-06   |34  |9501.0 |5         |
|15 |Wanda   |359-6973      |Nam.nulla.magna@In.org              |2005-08-21   |27  |1539.0 |5         |
|35 |Aurora  |1-865-751-3479|magna@Cras.net                      |2017-10-21   |54  |4588.0 |5         |
|50 |Ross    |1-587-285-1837|at.risus@milaciniamattis.c

#### La distinct()
Función PySpark se usa para eliminar / eliminar las filas duplicadas (todas las columnas) de DataFrame y dropDuplicates()se usa para eliminar filas según las columnas seleccionadas (una o varias).

#### 🧱 1️⃣ Usando la API de DataFrame (Spark Core)

In [50]:
from pyspark.sql.functions import col

# ============================================================
# DISTINCT (Eliminar filas completamente duplicadas)
# ============================================================

distinctDF = df.distinct()
print("Cantidad de registros distintos: " + str(distinctDF.count()))
distinctDF.show(truncate=False)

# ============================================================
# DROP DUPLICATES (equivalente a DISTINCT)
# ============================================================

df2 = df.dropDuplicates()
print("Cantidad de registros únicos: " + str(df2.count()))
df2.show(truncate=False)

# ============================================================
# DROP DUPLICATES en columnas específicas
# ============================================================
# Ejemplo: eliminar duplicados considerando solo NOMBRE y SALARIO
dropDisDF = df.dropDuplicates(["NOMBRE", "SALARIO"])
print("Cantidad de registros únicos por NOMBRE y SALARIO: " + str(dropDisDF.count()))
dropDisDF.show(truncate=False)

# Otro ejemplo: eliminar duplicados considerando empresa y salario
dropEmpSalDF = df.dropDuplicates(["ID_EMPRESA", "SALARIO"])
print("Cantidad de registros únicos por EMPRESA y SALARIO: " + str(dropEmpSalDF.count()))
dropEmpSalDF.show(truncate=False)

# ============================================================
# Ejemplo adicional: comparar el conteo antes y después
# ============================================================

total_original = df.count()
total_unicos = df.dropDuplicates().count()

print(f"Total original: {total_original}, Registros únicos: {total_unicos}")


Cantidad de registros distintos: 100
+---+---------+--------------+-------------------------------------------+-------------+----+-------+----------+
|ID |NOMBRE   |TELEFONO      |CORREO                                     |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+-------------------------------------------+-------------+----+-------+----------+
|89 |Kelly    |430-9134      |turpis.In@egestasFusce.ca                  |2016-04-01   |55  |10110.0|6         |
|57 |Jillian  |1-286-285-0336|ipsum@luctus.org                           |2006-06-02   |47  |13445.0|8         |
|97 |Flavia   |1-559-270-7164|erat.vel.pede@sedtortor.co.uk              |2004-11-14   |27  |13473.0|3         |
|7  |Mark     |1-680-102-6792|Quisque.ac@placerat.ca                     |2006-04-21   |52  |8112.0 |5         |
|100|Cynthia  |148-9696      |justo.nec.ante@tinciduntvehicula.org       |2008-01-23   |57  |8682.0 |5         |
|27 |Alexander|912-0676      |semper.auctor.Mauris@mollisle

#### 🧩 2️⃣ Usando Spark SQL

In [51]:
# Registrar vista temporal
df.createOrReplaceTempView("empleados")

# ============================================================
# DISTINCT (elimina filas duplicadas completas)
# ============================================================

spark.sql("""
  SELECT DISTINCT *
  FROM empleados
""").show(truncate=False)

# ============================================================
# DISTINCT sobre columnas específicas
# ============================================================

# Ejemplo: obtener combinaciones únicas de NOMBRE y SALARIO
spark.sql("""
  SELECT DISTINCT NOMBRE, SALARIO
  FROM empleados
""").show(truncate=False)

# Otro ejemplo: obtener empresas y sus salarios únicos
spark.sql("""
  SELECT DISTINCT ID_EMPRESA, SALARIO
  FROM empleados
""").show(truncate=False)

# ============================================================
# Verificar conteos con SQL
# ============================================================

spark.sql("""
  SELECT COUNT(*) AS total_original FROM empleados
""").show()

spark.sql("""
  SELECT COUNT(DISTINCT NOMBRE, SALARIO) AS total_unicos_nombre_salario
  FROM empleados
""").show()


+---+---------+--------------+-------------------------------------------+-------------+----+-------+----------+
|ID |NOMBRE   |TELEFONO      |CORREO                                     |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+-------------------------------------------+-------------+----+-------+----------+
|89 |Kelly    |430-9134      |turpis.In@egestasFusce.ca                  |2016-04-01   |55  |10110.0|6         |
|26 |Brenden  |1-455-726-9413|elit.pede.malesuada@liberomaurisaliquam.org|2000-03-17   |33  |20549.0|7         |
|81 |Joy      |1-379-384-0646|mi.Aliquam@nislNulla.org                   |2015-11-15   |19  |1256.0 |2         |
|65 |Nehru    |142-9576      |nibh@sagittis.org                          |2017-08-13   |34  |12423.0|4         |
|62 |Amelia   |119-9184      |Donec.egestas.Aliquam@turpisvitae.co.uk    |2016-01-15   |35  |6042.0 |8         |
|15 |Wanda    |359-6973      |Nam.nulla.magna@In.org                     |2005-08-21   |27  |153